
# EDA-Only Cleaning Pipeline (No Train/Test, No NPZ)
Use this notebook to **clean and standardize** your raw tables for exploratory data analysis (EDA).  
It **does not** split train/test, encode features, or create NPZ files.


In [ ]:

# === CONFIG: set your file names and key columns here ===
# Put the three CSVs in the SAME folder as this notebook.
FILENAMES = {
    "main": "primary_table.csv",          # <- change to your main fact table
    "dim1": "dim_customers.csv",          # <- change to your first dimension table
    "dim2": "dim_regions.csv",            # <- change to your second dimension table
}

# Keys used to join tables (left joins from main)
JOIN_KEYS = {
    "dim1_on": ["customer_id"],           # keys shared by main and dim1
    "dim2_on": ["region_id"],             # keys shared by main and dim2
}

# Date columns present in the merged dataset (set only those you actually have)
DATE_COLS = ["dob", "open_date", "last_activity"]

# Columns that should be non-negative (clip at 0 if negative shows up)
NON_NEGATIVE_COLS = ["price", "amount", "quantity", "revenue", "num_vintage"]

# Optional: columns that are clear identifiers (skip winsorizing on these)
ID_LIKE_COLS = ["customer_id", "region_id", "order_id"]


In [ ]:

import pandas as pd
import numpy as np
from pathlib import Path

def normalize_col(c: str) -> str:
    if not isinstance(c, str): 
        return c
    return c.strip().lower().replace(" ", "_")

def winsorize_series(s: pd.Series, low=0.01, high=0.99):
    if s.isna().all():
        return s
    lo, hi = s.quantile([low, high])
    return s.clip(lower=lo, upper=hi)

print("✔ Imports ready")


In [ ]:

# === LOAD ===
base = Path(".")
paths = {k: base / v for k, v in FILENAMES.items()}
for k, p in paths.items():
    if not p.exists():
        print(f"⚠ File not found for '{k}': {p}. Update FILENAMES above.")
        
df_main = pd.read_csv(paths["main"])
d1 = pd.read_csv(paths["dim1"])
d2 = pd.read_csv(paths["dim2"])

print("Shapes:", { "main": df_main.shape, "dim1": d1.shape, "dim2": d2.shape })


In [ ]:

# === STANDARDIZE COLUMN NAMES ===
for d in (df_main, d1, d2):
    d.columns = [normalize_col(c) for c in d.columns]

print("Sample of standardized columns in main:", df_main.columns.tolist()[:10])


In [ ]:

# === MERGE (Left joins from main) ===
df = df_main.merge(d1, on=JOIN_KEYS["dim1_on"], how="left")
df = df.merge(d2, on=JOIN_KEYS["dim2_on"], how="left")
print("Merged shape:", df.shape)


In [ ]:

# === TYPE FIXES ===
# Parse dates
for c in DATE_COLS:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# Convert object columns that look numeric into numeric
obj_cols = df.select_dtypes("object").columns.tolist()
for c in obj_cols:
    # attempt numeric coercion; if most values become NaN, revert to original
    coerced = pd.to_numeric(df[c].str.replace(",","", regex=False), errors="coerce")
    if coerced.notna().mean() >= 0.6:  # at least 60% convertible
        df[c] = coerced

print("Dtypes after coercion:")
print(df.dtypes.head(20))


In [ ]:

# === DERIVED FIELDS (optional) ===
today = pd.Timestamp.today().normalize()

if "dob" in df.columns:
    age_years = ((today - df["dob"]).dt.days / 365.25).round()
    df["age"] = age_years.clip(lower=0, upper=120)
    # store as Int64 (nullable)
    df["age"] = df["age"].astype("Int64")

if "open_date" in df.columns:
    tenure_months = ((today - df["open_date"]).dt.days / 30.44).round(0)
    df["num_vintage"] = tenure_months.clip(lower=0)
    df["num_vintage"] = df["num_vintage"].astype("Int64")

print("Derived columns present:", [c for c in ["age","num_vintage"] if c in df.columns])


In [ ]:

# === MISSING VALUES ===
# Numeric: fill with median (overall)
num_cols = df.select_dtypes(include=[np.number, "Float64", "Int64"]).columns.tolist()
for c in num_cols:
    med = df[c].median(skipna=True)
    df[c] = df[c].fillna(med)

# Categorical: fill with "Unknown"
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    df[c] = df[c].astype("string").str.strip()
    df[c] = df[c].fillna("Unknown")


In [ ]:

# === INVALIDS & OUTLIERS ===
# Clip common non-negative fields
for c in NON_NEGATIVE_COLS:
    if c in df.columns:
        df[c] = df[c].clip(lower=0)

# Light winsorization for numeric columns (skip obvious IDs)
skip = set(ID_LIKE_COLS)
for c in num_cols:
    if c not in skip and df[c].nunique(dropna=True) > 5:
        df[c] = winsorize_series(df[c])

# Replace any remaining inf
df.replace([np.inf, -np.inf], np.nan, inplace=True)
for c in num_cols:
    med = df[c].median(skipna=True)
    df[c] = df[c].fillna(med)


In [ ]:

# === DEDUP & USELESS COLUMNS ===
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"Removed {before - after} duplicate rows.")

# Drop near-constant columns or those >95% missing (post-fill check is trivial; we check before-fill via an aux copy)
# (Here we approximate by checking variance / nunique)
to_drop = []
for c in df.columns:
    if df[c].nunique(dropna=True) <= 1:
        to_drop.append(c)

if to_drop:
    df.drop(columns=to_drop, inplace=True)
    print("Dropped near-constant columns:", to_drop)
else:
    print("No near-constant columns dropped.")


In [ ]:

# === SAVE CLEAN DATASET ===
out_path = Path("clean_dataset.csv")
df.to_csv(out_path, index=False)
print(f"✅ Saved: {out_path.resolve()}")
print("Preview:")
display(df.head(5))
print("Shape:", df.shape)
